# PREPROCESSING 




In [ ]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

In [ ]:
train = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [ ]:
U = train.pivot("userID", "movieID", "rating")
U.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,4.0,3.0,NaN,4.0,3.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
U.shape

(6040, 3705)

In [ ]:
train_size = 6030
train = U[:train_size]
test = U[train_size:]

In [ ]:
test

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6031,5.0,5.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
6032,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
6033,4.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6034,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6035,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,4.0,NaN,3.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**I did create a data for user to user predection and I also did create a small test to see if my model is working and my rmse results for this users(not best test approach but still usable for me to see if there is any problem)**

# User to user collaborative filtering method.

In [ ]:

myPreds = []
realLabels = []
for userID in test.index:
 for movieID in test.columns:
    if not np.isnan(test.loc[userID, movieID]):
      filtered_train = train.dropna(subset=[movieID])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if 20 < len(filtered_train):
        k=20
        nn = nei.NearestNeighbors(n_neighbors=k, metric='cosine')
        nn.fit(filtered_train)
        test_row = test.loc[userID:userID, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)
        indices = nn.kneighbors(test_row, return_distance=False)[0]
        userIds = filtered_train.iloc[indices, :].index
        pred = filtered_train.loc[userIds, movieID].values.mean()
        x=test.loc[userID:userID, :]
        x=x.mean(axis=1).values.reshape(-1,1)
        pred=pred+x
        if pred>5:
          pred=5
        myPreds.append(pred)
        real_label = test.loc[userID, movieID]
        realLabels.append(real_label)  
      else:
        k=len(filtered_train)
        if k==0:
          pred=3.5
        else:
         k=len(filtered_train)  
         nn = nei.NearestNeighbors(n_neighbors=k, metric='cosine')
         nn.fit(filtered_train)
         test_row = test.loc[userID:userID, :]
         test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
         test_row.fillna(0, inplace=True)

         indices = nn.kneighbors(test_row, return_distance=False)[0]

         userIds = filtered_train.iloc[indices, :].index
         pred = filtered_train.loc[userIds, movieID].values.mean()
         pred=pred+test_row.mean(axis=1).values.reshape(-1,1)  
         if pred>5:
          pred=5  
        myPreds.append(pred)
        real_label = test.loc[userID, movieID]
        realLabels.append(real_label)

**I will use K as 20(you stated that most of the time it is near 20 so I will start with it) and I will use cousine metric and if no information about UserId than prediction will be 3.5(to not get a error) and if len of filtered data lower than 20 it will  be new k. And as no prediction can be greater than 5 5 will be my max prediction.**

In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


0.692517504607774

In [ ]:
len(myPreds)

1193

**I did make predictions for 1193 pair and my rmse 0.69**

# CREATING KAGGLE SUBMISSION

In [ ]:
test = pd.read_csv("test_without_labels.csv")

In [ ]:
test = pd.read_csv("test_without_labels.csv")
test[['userID','movieID']] = test.IDs.str.split(pat="_",expand=True)

In [ ]:
test["rating"]=sample.rating

In [ ]:
test = test.pivot("userID", "movieID", "rating")


In [ ]:

myPreds = []
for i in test.IDs:
  a = i.split("_")
  userID=int(a[0])
  movieID=int(a[1])
  filtered_train = U.dropna(subset=[movieID])
  filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
  filtered_train.fillna(0, inplace=True)
  if 20 < len(filtered_train):
        k=20
        nn = nei.NearestNeighbors(n_neighbors=k, metric='cosine')
        nn.fit(filtered_train)
        test_row = U.loc[userID:userID, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)
        indices = nn.kneighbors(test_row, return_distance=False)[0]
        userIds = filtered_train.iloc[indices, :].index
        pred = filtered_train.loc[userIds, movieID].values.mean()
        x=U.loc[userID:userID, :]
        x=x.mean(axis=1).values.reshape(-1,1)
        pred=pred+x
        if pred>5:
          pred=5
        myPreds.append(pred)
  else:
        k=len(filtered_train)
        if k==0:
          pred=3.5
        else:
         k=len(filtered_train)  
         nn = nei.NearestNeighbors(n_neighbors=k, metric='cosine')
         nn.fit(filtered_train)
         test_row = U.loc[userID:userID, :]
         test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
         test_row.fillna(0, inplace=True)
         indices = nn.kneighbors(test_row, return_distance=False)[0]
         userIds = filtered_train.iloc[indices, :].index
         pred = filtered_train.loc[userIds, movieID].values.mean()
         x=U.loc[userID:userID, :]
         x=x.mean(axis=1).values.reshape(-1,1)
         pred=pred+x
         if pred>5:
          pred=5  
        myPreds.append(pred)
  print(pred)  

Görüntülenen çıkış son 5000 satıra kısaltıldı.
[[2.93411412]]
5
[[4.23520021]]
[[2.71028624]]
[[4.31800327]]
[[3.11363552]]
[[4.64131965]]
[[2.98857046]]
[[4.74875085]]
[[4.64221018]]
[[3.99150438]]
[[4.00917205]]
[[3.89298404]]
[[3.53944112]]
[[4.40574875]]
[[4.21234662]]
[[3.62280784]]
[[4.2827547]]
[[2.89769291]]
[[3.47453205]]
[[3.47089818]]
[[4.40486295]]
[[4.57020909]]
[[4.19039417]]
[[3.14278969]]
[[2.60827938]]
[[2.74521631]]
[[2.94751116]]
[[4.26777855]]
[[3.55757758]]
[[2.81972562]]
[[3.75939293]]
[[3.24911915]]
5
[[4.27489544]]
[[4.19978733]]
[[3.49375373]]
[[4.1385161]]
[[1.8017553]]
[[3.31232985]]
[[3.70125089]]
[[3.00222297]]
[[3.81866333]]
[[3.84737093]]
[[3.56952058]]
[[2.33983274]]
[[3.53064634]]
[[4.71235271]]
[[4.68477686]]
[[2.73132425]]
[[4.19980592]]
[[3.41490454]]
[[3.24641613]]
[[3.62559988]]
[[2.93290456]]
[[3.05182184]]
[[3.51729339]]
5
[[4.14549548]]
[[2.28711268]]
[[4.36817574]]
[[2.10387866]]
[[4.13817612]]
[[2.33491697]]
[[3.41729163]]
[[4.30641946]]
[[3.8

**I did use my code on caggle data to create my predictions**

In [ ]:
myPreds_num=[]

In [ ]:
for i in myPreds:
  i=float(i)
  myPreds_num.append(i)
 

In [ ]:
test["rating"]=myPreds_num

In [ ]:

dfpred = pd.DataFrame(test, columns = ['IDs', "rating"])
dfpred.to_csv("output.xlsx",index=False)

**My result for caggle is 0.89047**